In [1]:
import numpy as np
import matplotlib.pyplot as plt
import folium
from ipywidgets import interact
import pandas as pd
from shapely import intersection
from shapely.geometry import MultiPoint, Polygon, Point
from shapely import voronoi_polygons

def random_color():
    color = np.random.randint(16, 256, size=3)
    color = [str(hex(i))[2:] for i in color]
    return '#' + ''.join(color).upper()

def create_point(row):
    return Point(row['LATITUDE'], row['LONGITUDE'])

In [2]:
data = pd.read_csv("bike_share_data/cleaned_data.csv")
data['point'] = data.apply(create_point, axis=1)

### copy of class code only for debuging purposes

In [31]:
import pandas as pd
from shapely import intersection
from shapely.geometry import MultiPoint, Polygon, Point
from shapely import voronoi_polygons

class weightedVoroniDigram:
    """
    Class that computes weighted voronoi digrams for our use case
    
    """
    def __init__(self, df):
        """
        """
        self.data = df

    def find_optimal_voronoi_digram(num_to_add, num_to_remove, df, boundary):
        """
        Args:
            num_to_add (int): the number of points to add to data
            num_to_remove (int): number of points in dataset to remove
            df (dataframe): dataframe with a column for "weights", "station_name", and the "point" (x,y).
            boundary (shapley polygon): Bound the Vorinoi digram to this region.
        """
        
        removed_df = weightedVoroniDigram.__find_optimal_point_to_remove(num_to_remove, df, boundary)
        added_removed_df = weightedVoroniDigram.__find_optimal_points_to_add(num_to_add, removed_df, boundary)

        
        
            
        return weightedVoroniDigram.__weighted_voronoi_polygons(added_removed_df, boundary)
        

    def __find_optimal_point_to_remove(num_to_remove, df, boundary):
        """
        Args:
            num_to_remove (int): number of points in dataset to remove
            df (dataframe): dataframe with a column for "weights", "station_name", and the "point" (x,y).
            boundary (shapley polygon): Bound the Vorinoi digram to this region.

        return, dataframe of with the num_to_remove points removed that were the worst locations
        """
        return df

    def __find_optimal_points_to_add(num_to_add, df, boundary):
        """
        Probably use regression to compute the anticipated weights for the new points to be added.

        
        Args:
            num_to_add (int): the number of points to add to data
            df (dataframe): dataframe with a column for "weights", "station_name", and the "point" (x,y).
            boundary (shapley polygon): Bound the Vorinoi digram to this region.

        return, dataframe of with the num_to_add points added that were the best locations
        """
        return df
        
    def __weighted_voronoi_polygons(df, boundary):
        """
        Given a dataframe with weights compute the shapley poloygons for the weighted voronoi digram on the pts.
        
        Args:
            df (dataframe): dataframe with a column for "weights", "station_name", and the "point" (x,y)
            boundary (shapley polygon): Bound the Vorinoi digram to this region.
        Returns:
            dataframe: with the columns "poly", column of Shapley Poloygons that is the voronoi digram for the point, 
                                                    "weights", "station_name", and the "point" (x,y)
        """
        points2 =  MultiPoint(df['point'].to_list())
       
        voronoi_diagram = voronoi_polygons(points2)

        vor_polygons = []
        for geom in voronoi_diagram.geoms:
            xx, yy = geom.exterior.coords.xy
            coord = list(zip(xx, yy))
            inter = intersection(geom, boundary)
            xx, yy = inter.exterior.coords.xy
            coord_clipped = list(zip(xx, yy))
            
            vor_polygons.append(coord_clipped)

        df['poly'] = vor_polygons
        return df



### Voronoi digram plotter

In [32]:
# from voronoi import weightedVoroniDigram

In [37]:
def generate_points(n):
    return data[['LATITUDE','LONGITUDE']].to_numpy()
    
def plot_voronoi(df):
    # Create Folium map
    points = df[['LATITUDE','LONGITUDE']].to_numpy()
    map_center = points.mean(axis=0)
    m = folium.Map(location=map_center, zoom_start=10)

    for index, row in df.iterrows():
        folium.Polygon(
                    locations=row['poly'],
                    color="blue",
                    weight=4,
                    fill_color=random_color(),
                    fill_opacity=0.5,
                    fill=True,
                    popup=f"{row['station_name']}",
                    ).add_to(m)
        
        folium.CircleMarker(location=[row['LATITUDE'],row['LONGITUDE']],
                        radius=2,
                        weight=2).add_to(m)
    
    # Display map
    display(m)

@interact(Add=(0, 10), Remove=(0,10))
def adjust_points(Add, Remove):
    boundary = Polygon([(38.7, -76.8), (38.7,-77.4), (39.2,-77.4), (39.2,-76.8)])
    points = weightedVoroniDigram.find_optimal_voronoi_digram(Add, Remove, data, boundary)
    # points = generate_points(Add)
    plot_voronoi(points)


interactive(children=(IntSlider(value=5, description='Add', max=10), IntSlider(value=5, description='Remove', …